OLD NOTEBOOK: SEE osm_pbf_power_data_extractor.py which does everything.

In [1]:
import os, sys, time
#IMPORTANT: RUN SCRIPT FROM THIS SCRIPTS DIRECTORY i.e data_exploration/ TODO: make more robust
##os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../../scripts')
from iso_country_codes import AFRICA_CC



import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import geoplot
import matplotlib.pyplot as plt
from iso_country_codes import AFRICA_CC
from osm_pbf_power_data_extractor import convert_pd_to_gdf_lines, convert_pd_to_gdf

import logging
logger = logging.getLogger(__name__)

# SUBSTATIONS

In [3]:
#----------- SUBSTATIONS -----------
# Load uncleaned data
df_all_substations = pd.read_csv(os.getcwd()+"/data/raw/africa_all_raw_substations.csv")
# Clean
df_all_substations = df_all_substations.reset_index(drop=True)
df_all_substations = df_all_substations.dropna(subset=['tags.voltage']) # Drop any substations with Voltage = N/A

#~ Generate Files

#CSV
# outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_substations'+'_cleaned.')
# df_all_substations.to_csv(outputfile_partial + 'csv') # Generate CSV

#GeoJSON
##gdf_substations = convert_pd_to_gdf(df_all_substations)
##gdf_substations.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


#display(df_all_substations)

# LINES 

### Check old unique values

In [298]:
# Load pypsa-eur data
df_all_lines = (pd.read_csv(os.getcwd()+"/entsoegridkit/lines.csv", quotechar="'", true_values='t', false_values='f',
                         dtype=dict(line_id='str', bus0='str', bus1='str',
                                    underground="bool", under_construction="bool")).set_index('line_id').rename(columns=dict(voltage='v_nom', circuits='num_parallel')))

#print(df_all_lines.geometry.unique())
display(df_all_lines)

,bus0,bus1,v_nom,num_parallel,length,underground,under_construction,tags,geometry
line_id,,,,,,,,,
6568,33,34,132,1,25687.871153,False,False,"""MW""=>""0.0"", ""TSO""=>"" "", ""oid""=>""51000"", ""EIC_...","LINESTRING(32.2819899090672 26.2326704525422,3..."
11675,31,34,132,1,88419.785937,False,False,NaN,"LINESTRING(32.3037477094187 26.0016233689967,3..."
6571,37,40,132,1,26394.698842,False,False,"""MW""=>""0.0"", ""TSO""=>"" "", ""oid""=>""51003"", ""EIC_...","LINESTRING(32.6501207538248 26.344729862945,32..."
6569,34,40,132,1,43972.508007,False,False,"""MW""=>""0.0"", ""TSO""=>"" "", ""oid""=>""51001"", ""EIC_...","LINESTRING(32.7250972033021 26.1158696376196,3..."
6570,40,42,132,1,23509.128839,False,False,"""MW""=>""0.0"", ""TSO""=>"" "", ""oid""=>""51002"", ""EIC_...","LINESTRING(32.7259342990352 26.1158221410256,3..."
...,...,...,...,...,...,...,...,...,...
12574,6885,8002,750,1,149251.722487,False,False,NaN,"LINESTRING(30.8701295047876 47.1281577300625,3..."
11495,3065,8002,750,1,49796.609048,False,True,NaN,"LINESTRING(30.2608839547144 47.295882632508,30..."
12573,8002,8424,750,1,90661.434110,False,False,NaN,"LINESTRING(29.956055 46.6076264806737,29.95971..."


# Import data and create final dataframe layout

In [299]:
# Load raw cable data
df_cables = gpd.read_file(os.getcwd()+"/data/raw/africa_all_raw_cables.geojson") 

# Modification - create final dataframe layout
df_cables = df_all_cables.rename(
    columns = {
        "id": "line_id",
        "tags.voltage": "voltage",
        "tags.circuits": "circuits",
        "tags.cables": "cables",
        "tags.frequency": "tag_frequency",
        "tags.power": "tag_type",
        "tags.location": "tag_location",
        "lonlat": "geometry",
        "Country": "country",  # new/different to PyPSA-Eur
        "Length": "length",
    }
)

# Add NaN as default
df_cables["bus0"] = np.nan
df_cables["bus1"] = np.nan
#df_all_cables["length"] = np.nan # Now in dataset
df_cables["underground"] = np.nan
df_cables["under_construction"] = np.nan

#Rearrange columns
clist = ["line_id","bus0","bus1","voltage","circuits","length","underground",
         "under_construction","tag_type","tag_frequency", "tag_location","geometry", "country"]
df_cables = df_cables[clist]

# make float to integer
df_cables["line_id"] = df_cables["line_id"].astype(int)


#display(df_cables)
#df_all_cables[df_all_cables['tag_location']== "overground"]
#df_all_cables["tags.location"].unique()

In [300]:
# Load raw line data
df_lines = gpd.read_file(os.getcwd()+"/data/raw/africa_all_raw_lines.geojson") 

# Modification - create final dataframe layout
df_lines = df_lines.rename(
    columns = {
        "id": "line_id",
        "tags.voltage": "voltage",
        "tags.circuits": "circuits",
        "tags.cables": "cables",
        "tags.frequency": "tag_frequency",
        "tags.power": "tag_type",
        "lonlat": "geometry",
        "Country": "country",  # new/different to PyPSA-Eur
        "Length": "length",
    }
)

# Add NaN as default
df_lines["bus0"] = np.nan
df_lines["bus1"] = np.nan
#df_all_lines["length"] = np.nan # commented because, we have now length data
df_lines["underground"] = np.nan
df_lines["under_construction"] = np.nan

#Rearrange columns
clist = ["line_id","bus0","bus1","voltage","circuits","length","underground",
         "under_construction","tag_type","tag_frequency", "cables","geometry", "country"]
df_lines = df_lines[clist]

#display(df_all_lines)

### Combine cable and line to one  "df_all_lines" dataset

In [301]:
df_all_lines = pd.concat([df_lines,df_cables])
# df_all_lines

### Define underground, under_construction information, frequency, circuits

In [302]:
# under construction
df_all_lines["under_construction"] = False # default. Not more information atm available

# underground
df_all_lines["underground"] = (df_all_lines["tag_type"] == "cable") # Simplified. If tag_type cable then underground is True. 
# More information extractable for "underground" by looking at "tag_location".
if 'tag_location' in df_all_lines: # drop column if exist
    df_all_lines.drop(columns = "tag_location", inplace=True)

# frequency
df_all_lines["tag_frequency"] = 50
#df_all_lines["tag_frequency"].unique()

# circuits
if df_all_lines["cables"].dtype != int: # if not int make int
    df_all_lines.loc[(df_all_lines["cables"] < "3") | df_all_lines["cables"].isna(), "cables"] = "0" #HERE. "0" if cables "None", "nan" or "1"
    df_all_lines["cables"] = df_all_lines["cables"].astype("int")
if 4 or 5 in df_all_lines["cables"].values: # downgrade 4 and 5 cables to 3... 
    # Reason: 4 cables have 1 lighting protection cables, 5 cables has 2 LP cables; see https://hackaday.com/2019/06/11/a-field-guide-to-transmission-lines/
    df_all_lines.loc[(df_all_lines["cables"] == 4) | (df_all_lines["cables"] == 5), "cables"] = 3 # where circuits are "0" make "1"
df_all_lines.loc[df_all_lines["circuits"].isna(), "circuits"] = df_all_lines.loc[df_all_lines['circuits'].isna(), "cables"] / 3 # one circuit contains 3 cables
df_all_lines["circuits"] = df_all_lines["circuits"].astype(int)
df_all_lines.loc[(df_all_lines["circuits"] == "0") | (df_all_lines["circuits"] == 0), "circuits"] = 1 # where circuits are "0" make "1"

# df_all_lines["circuits"].unique()
# df_all_lines["cables"].unique()
# display(df_all_lines)

### Clean voltage

In [303]:
# Drop any row with Voltage = N/A
df = df_all_lines.dropna(subset=['voltage']) 

#Split semicolon separated cells i.e. [66000;220000] and create new identical rows
lst_col = 'voltage'
x = df.assign(**{lst_col:df[lst_col].str.split(';')})
x = pd.DataFrame({
    col:np.repeat(x[col].values, x[lst_col].str.len())
    for col in x.columns.difference([lst_col])
    }).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]
df_all_lines = x

#display(df_all_lines)

In [304]:
# Create unique line_id's
# The steps below create unique line_id's without loosing the original OSM line_id 

# Context
# The previous duplication of rows (to split the voltage) lead to a couple of same line_id (about 30% of dataset)

# Method
# Unique line_id are created by simply adding -1,-2,-3 to the original line_id
# Every unique id gets a -1 
# If a line_id exist i.e. three times it it will the counted by cumcount -1,-2,-3 making the id unique

if df_all_lines["line_id"].count() != df_all_lines["line_id"].nunique(): # operate only if line_id is not already unique (nunique counts unique values)
    df_all_lines["cumcount"] = df_all_lines.groupby(["line_id"]).cumcount() # create cumcount column. Cumcount counts 0,1,2,3 the number of duplicates
    df_all_lines["cumcount"] = df_all_lines["cumcount"] + 1 # avoid 0 value for better understanding
    df_all_lines["line_id"] = df_all_lines["line_id"].astype(str) + "-" + df_all_lines["cumcount"].values.astype(str) # add cumcount to line_id to make line_id unique
    df_all_lines.drop(columns = "cumcount", inplace=True) # remove cumcount column

#display(df_all_lines)

In [311]:
# Remove all non-numeric values

df_all_lines['voltage'] = df_all_lines['voltage'].apply(lambda x: pd.to_numeric(x, errors='coerce')).astype(float) # if cell can't converted to float -> nan
df_all_lines = df_all_lines.dropna(subset=['voltage']) # Drop any row with Voltage = N/A
df_all_lines.loc[:,"voltage"]  = df_all_lines['voltage'].astype(int)
#df_all_lines['voltage'].unique()

array([220000, 400000,  66000,  60000,  30000, 225000,  90000,   3000,
       150000,  22000, 132000, 330000, 110000, 350000, 161000,  70000,
       120000, 500000,      0,  11000,  63000, 230000,  45000, 115000,
        88000,     30,  69000,  25000, 138000,  33000,  20000,    400,
        22500, 533000, 275000,  19000,     11,     33, 765000,  44000,
       380000,    220,  15000,    230,    240,    480,    180])

In [312]:
# Keep rows with x > 110 kV as it is considered as transmission level

df_all_lines = df_all_lines[df_all_lines.voltage > 110000]

In [316]:
display(df_all_lines)
display(df_all_lines['voltage'].unique())

,line_id,bus0,bus1,voltage,circuits,length,underground,under_construction,tag_type,tag_frequency,cables,geometry,country
0,380986751-1,NaN,NaN,220000,1,40467.295973,False,False,line,50,3,"LINESTRING (-1.83638 34.78691, -1.83590 34.789...",algeria
1,173682341-1,NaN,NaN,400000,1,168.067783,False,False,line,50,3,"LINESTRING (-0.90176 35.08983, -0.90181 35.090...",algeria
2,655093026-1,NaN,NaN,220000,1,64.987785,False,False,line,50,3,"LINESTRING (0.14621 34.90905, 0.14639 34.90922...",algeria
5,383315204-1,NaN,NaN,220000,1,98637.025052,False,False,line,50,3,"LINESTRING (3.38017 36.71034, 3.38385 36.71236...",algeria
8,655093020-1,NaN,NaN,220000,1,110.799435,False,False,line,50,3,"LINESTRING (0.14661 34.90939, 0.14672 34.90930...",algeria
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15178,456151088-1,NaN,NaN,132000,1,131.900676,True,False,cable,50,0,"LINESTRING (31.01196 -29.81361, 31.01196 -29.8...",south-africa
15179,668505779-1,NaN,NaN,132000,1,36903.154434,True,False,cable,50,0,"LINESTRING (39.19950 -6.63733, 39.19991 -6.637...",tanzania
15180,445315079-1,NaN,NaN,150000,1,948.419880,True,False,cable,50,0,"LINESTRING (9.97671 37.21045, 9.96819 37.21059)",tunisia
15263,696726292-1,NaN,NaN,132000,1,155.359211,True,False,cable,50,0,"LINESTRING (33.13385 0.50202, 33.13392 0.50195...",uganda


array([220000, 400000, 225000, 150000, 132000, 330000, 350000, 161000,
       120000, 500000, 230000, 115000, 138000, 533000, 275000, 765000,
       380000])

In [131]:
## Generate Files
## CSV
# outputfile_partial = os.path.join(os.getcwd(), 'data', 'africa_all'+'_lines'+'_cleaned.')  
# df_all_lines.to_csv(outputfile_partial + 'csv')  # Generate CSV
## GeoJSON
# gdf_lines = convert_pd_to_gdf_lines(df_all_lines, simplified=True)
# gdf_lines.to_file(outputfile_partial+'geojson',
#             driver="GeoJSON")  # Generate GeoJson


# GENERATORS

In [5]:
# ----------- Generator -----------

#Load uncleaned data
df_all_generators = pd.read_csv(os.getcwd()+"/data/raw/africa_all_raw_generators.csv")

#Clean data
df_all_generators = df_all_generators.reset_index(drop=True)
df_all_generators = df_all_generators[df_all_generators['tags.generator:output:electricity'].astype(str).str.contains('MW')] #removes boolean 
df_all_generators['tags.generator:output:electricity'] = df_all_generators['tags.generator:output:electricity'].str.extract('(\d+)').astype(float)
df_all_generators = df_all_generators.rename(columns = {'tags.generator:output:electricity':"power_output_MW"})


## Generate Files

#CSV
#outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_generators'+'_cleaned.')
#df_all_generators.to_csv(outputfile_partial + 'csv') # Generate CSV

#GeoJSON
# gdf_generators = convert_pd_to_gdf(df_all_generators)
# gdf_generators.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


#display(df_all_generators)